### MULTIPLE RUNS: Import Libraries, READ SWMM INP file, Setup MongoDB Database to store results

In [9]:
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import *
from pymongo import MongoClient
from random import randint

# Constants:
squareFeetInOneAcre = 43560.0
# Specify important files and parameters for the location:
swmmInpFileNameStr = "wingohocking_PWD_2012-13.inp"  # initial SWMM .inp file
treatmentRatio = 0.081   # cfs/imperv_acre calibrated for Wingohocking T-14 outfall 7/1/12 - 6/30/13
lidDictFileNameStr = "LID_Dict.yaml"
lidShedsFileNameStr = "Wingohocking_LID_Sheds_List.yaml"
# Calculate parameters for the watershed from the SWMM model INP file:
watershedImpervAreaAcre = total_impervious_area(swmmInpFileNameStr)
# Load YAML files 
f = open(lidDictFileNameStr,'r')   
lidDict = yaml.load(f)           # Read the LID ContribImpervArea and GreenedAcres
f.close()
f = open(lidShedsFileNameStr,'r')   
lidShedsList = yaml.load(f)  # Read the list of subcatchments where LIDs can be installed
f.close()
# Compute a dictionary of contributing impervious ACRES for each LID 
contribImpervAreaAcre = {}
for lid in lidDict:
    contribImpervAreaAcre[lid] = lidDict[lid]['ContribImpervArea'] / squareFeetInOneAcre  # acres
# Open the specified SWMM zeroLID INP file and obtain necessary data from it for multiple runs:    
swmmInpStr = read_inp_file(swmmInpFileNameStr)
(section_names,sections) = read_inp(swmmInpStr)
model = swmm_model("Model",section_names,sections)
subcatAreaDict = {}            # Total area in each subcatchment (never changes)
subcatImpPctDict = {}          # ORIGINAL Impervious percentage in each subcatchment (changes when LID added)
subcatTreatableAreaDict = {}   # Original IMPERVIOUS area in zero LID SWMM file
subcatMaxLID = {}              # Maximum number of each LID that can be installed in each subcatchment
subcatMaxGI = {}               # Maximum greened acres for each LID that can be installed in each subcatchment
# Now calculate the dictionaries defined above from data in the zero LID SWMM input file:
for lidShed in lidShedsList:
    area = model.getSubcatchmentArea(lidShed)
    subcatAreaDict[lidShed] = area
    pctImperv = model.getSubcatchmentPctImperv(lidShed)
    subcatImpPctDict[lidShed] = pctImperv
    subcatTreatableAreaDict[lidShed] = area*pctImperv/100.0
    subcatMaxLID[lidShed] = {}
    for lid in lidDict:
        subcatMaxLID[lidShed][lid] = int(subcatTreatableAreaDict[lidShed]/contribImpervAreaAcre[lid])
# Specify MongoDB database server to use and login with credentials:    
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collectionName = 'y17m04d24_45_Subcat_wakefield_BR_RG_Only'
thisRunCollection = db[collectionName]    # use or create the collection
print subcatMaxLID['TT14-574_shed']['MorrisTT']

121


### Execute the Multirun Case for a single specified LID after Zero Run Case is run BELOW

In [12]:
lid = 'wakefield_BR_RG'
lidFractList = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]  # Fraction of the maximum number of LID in each subcatchment
for lidFract in lidFractList:
    for lidShed in lidShedsList:
        numLID = int(lidFract*subcatMaxLID[lidShed][lid])
        model.lidChangeNumber(lidShed,lid,numLID)
        area = model.getLidArea(lidShed,lid)   # the lid footprint in the subcatchment (all subcats the same now)
        model.lidChangeArea(lidShed,lid,area,contribImpervAreaAcre[lid])  # must call to properly set FromImp parameter !!
    swmmInputFileStr=model.output()
    swmmRptDict = swmm_run_return_results(swmmInputFileStr)
    docId = thisRunCollection.insert_one(swmmRptDict).inserted_id
    print "Stored record %s for %0.2f fraction of each subcatchment's treatable area" % \
          (docId,lidFract)

3.0 min, 5.51 sec
Stored record 58fdda88c101574830c3b79e for 0.10 fraction of each subcatchment's treatable area
3.0 min, 3.08 sec
Stored record 58fddb40c101574830c3b79f for 0.20 fraction of each subcatchment's treatable area
3.0 min, 3.08 sec
Stored record 58fddbf9c101574830c3b7a0 for 0.30 fraction of each subcatchment's treatable area
2.0 min, 58.50 sec
Stored record 58fddcacc101574830c3b7a1 for 0.40 fraction of each subcatchment's treatable area
2.0 min, 59.47 sec
Stored record 58fddd61c101574830c3b7a2 for 0.50 fraction of each subcatchment's treatable area
2.0 min, 57.77 sec
Stored record 58fdde14c101574830c3b7a3 for 0.60 fraction of each subcatchment's treatable area
3.0 min, 3.38 sec
Stored record 58fddecdc101574830c3b7a4 for 0.70 fraction of each subcatchment's treatable area
3.0 min, 8.21 sec
Stored record 58fddf8bc101574830c3b7a5 for 0.80 fraction of each subcatchment's treatable area
3.0 min, 9.70 sec
Stored record 58fde04ac101574830c3b7a6 for 0.90 fraction of each subcatchme

## RUN ONLY ONCE:  the Zero LID case

In [10]:
swmmInpFileStr = read_inp_file(swmmInpFileNameStr)
swmmRptDict = swmm_run_return_results(swmmInpFileStr)
zeroLidDocId = thisRunCollection.insert_one(swmmRptDict).inserted_id
print "Stored record %s for zero LID usage" % zeroLidDocId

3.0 min, 4.82 sec
Stored record 58fdd8ffc101574830c3b79d for zero LID usage


### Display Zero LID case

In [11]:
# Show all collections in the database:
# collectionNames = sorted(db.collection_names())
# collectionNames
runs = db[collectionName]
cursor = runs.find()
noLidRun = cursor[0]
print noLidRun['peakCFS']
print noLidRun['precipIN']
print noLidRun['runoffIN']

4109.36
43.97
25.592


### Old SWMMWise code below

In [ ]:
# Run or continue a run:
%autoreload
import yaml
from swmm_run import *
from swmm_objects import *
from swmm_read import *
from copy import deepcopy

from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
# mongoServer = None   # use the local mongodb server installed on your computer
#mongoServer = 'mongodb://server.mcgarity.info:27017/'  # Art's personal server
#mongoServer = 'mongodb://ubuntu:27017/'  # ubuntu Linux virtual machine on Art's Mac
mongoServer = 'mongodb://aws-2.greenphilly.net:27017/'  # aws-2 server for GreenPhilly
database = "jcohen4"
username = "jcohen4"
password = "wingohocking"
authenticationDatabase = "jcohen4"
swmmInpFile = "wingohocking_PWD_Wakefield.inp"  # initial SWMM .inp file
lidListFile = "PWD_wingo_Wakefield_LID_List.yaml"
################################################
squareFeetInOneAcre = 43560.0
client = MongoClient(mongoServer)
db = client[database]   # use or create the database
db.authenticate(username, password, source=authenticationDatabase)  # pass credentials to database
collection = 'y17m01d10_AEM_45_Subcat_Wakefield_Rain_Garden_1'
thisRunCollection = db[collection]
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
(section_names,sections) = read_inp(swmmInpStr)
modelTemp = swmm_model("ModelTemp",section_names,sections)
f = open(lidListFile,'r')   # Read the LID parameters used to calculate [LID_USAGE]
lidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
#########################
numSubcatch = 45
numLID = 1
numberOfSimulations = 50
#########################
numVariables = numSubcatch*numLID
initialRemainingTreatableArea = {}   # Dict indexed by subcatchment name string 
for lidListEntry in range(numVariables):
    thisLidInfo = lidList[lidListEntry]
    subcatName = thisLidInfo['Subcat']
    areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres   
    impPctSubcat = modelTemp.getSubcatchmentPctImperv(subcatName)        
    initialRemainingTreatableArea[subcatName] = (impPctSubcat/100.0)*areaSubcat  # INITIAL values (Acre)
#print initialRemainingTreatableArea
permList = []
greenedAcreList = []

for i in range(0,numberOfSimulations):#number of simulations
    remainingTreatableArea = dict(initialRemainingTreatableArea)   # Dict indexed by subcatchment name string  
    lidListEntry = 0
    randomPerm = []
    greened = []
    lidListEntry = 0    
    for lid in range(numLID):
        for subcat in range(numSubcatch):
            thisLidInfo = lidList[lidListEntry]
            #lidName = thisLidInfo['LID']
            subcatName = thisLidInfo['Subcat']
            areaSubcat = modelTemp.getSubcatchmentArea(subcatName)     # Acres          
            contribImpervArea = thisLidInfo['ContribImpervArea']/squareFeetInOneAcre  #ft2 to Acres        
            numLidMaximum = int(remainingTreatableArea[subcatName]/contribImpervArea)
            #randomIntegerUpper = randint(1,19)
            #upperFraction = float(randomIntegerUpper)/20.0

            #randomScaleFactor = randint(3,50)
            #random1 = randint(1,randomScaleFactor)
            #random2 = randint(1,randomScaleFactor)
            #if random1 > random2:
            #    randomUpper = random1
            #    randomLower = random2
            #else:
            #    randomUpper = random2
            #    randomLower = random1
            ############
            #lower = numLidMaximum/randomUpper
            #upper = numLidMaximum/randomLower 
            ############
            fixedLowerFraction = 0.7
            fixedUpperFraction = 0.98
            lower = int(fixedLowerFraction*numLidMaximum)
            upper = int(fixedUpperFraction*numLidMaximum)
            ############
            randomNumberLid = randint(lower,upper)
            randomPerm.append(randomNumberLid)
            greened.append(randomNumberLid*lidList[lidListEntry]['GreenedAcres'])
            remainingTreatableArea[subcatName] = remainingTreatableArea[subcatName] - \
                  randomNumberLid*contribImpervArea #Acres
            lidListEntry += 1
        #print remainingTreatableArea
    permList.append(randomPerm)
    greenedAcreList.append(greened)
#print permList 
#print greenedAcreList


runNum = 0
for perm in permList:
    greened = greenedAcreList[runNum]
    totGreenedAcres = sum(greened)
    runNum += 1
    print "Run Number %s" % runNum  
    print "Total Greened Acres %0.2f" % totGreenedAcres
    print perm
    newRunList=deepcopy(lidList)  # copy the zeroLidList into a new list
    #totalAreaTreated = 0
    for i in range(0,numVariables): 
        #lidNumber = perm[i] # random number of LIDs for this variable
        #subArea = newRunList[i]['SubcatArea'] #area of subcatchment
        newRunList[i]['Number'] = perm[i]  # random number of lids for this variable
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection,perm)
    print "stored this run in mongodb DocId = %s (zero means failed SWMM run)" % newRunDocId
    #cursor = thisRunCollection.find_one({'_id':newRunDocId})

    #print "Stored record %s for volume = %s" % (newRunDocId,newVolume)
    #print "volumeReduction = %s" % (volumeReduction)
print "FINISHED ALL RUNS"
